In [1]:
%autosave 10

Autosaving every 10 seconds


In [3]:
#Testing the autosave function

In [5]:
#Now this is testing the automatic github pushing, is this working?

In [7]:
#Yes it is! Yay!

In [9]:
#Restarted the kernel, does it work now?

In [ ]:
#It does, into bash, you have to type the following: "python auto_commit.py"

### Continuing ML Tutorials and learning how to autosave on Github

### (That is what the above functions are showing)

# Now, we will do Neural Network for Cancer Prediction

## This tutorial is devised from Vappiah's Bioinformatics Projects for learning ML

### We will use RNA-seq data that was processed for differential gene expression in cancer types. The data we use is from https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq and will be a tabular data set

### The rows are cancer samples and the columns and gene counts, the last column is cancer categories

In [18]:
#We need the following libraries: 
#For data handling
import pandas as pd
import numpy as np
#